In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [2]:
loader = PyPDFLoader('D:/AI enabled Test case gen/Documents/Itinerary.pdf')

In [3]:
documents = loader.load()
# type(documents)
# print(documents)

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunk = splitter.split_documents(documents)
print(chunk)

[Document(metadata={'producer': 'Skia/PDF m131', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36', 'creationdate': '2024-11-23T13:06:00+00:00', 'title': 'Itinerary', 'moddate': '2024-11-23T13:06:00+00:00', 'source': 'D:/AI enabled Test case gen/Documents/Itinerary.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='PNR/Booking Reference LTQ7RI Confirmed Payment StatusComplete\nPassenger Information\nMr Twishanu Aich Roy Adult\nSector Seat 6E Add-ons\nCCU-BOM - -\nBOM-CCU - -\nKolkata\nCCU - Netaji Subhash Chandra Bose\nInternational Airport\n06:15 hrs, 10 Jan 2025\nMumbai\nBOM - Chhatrapati Shivaji Maharaj\nInternational Airport (Terminal 2)\n09:15 hrs, 10 Jan 2025\nMumbai\nBOM - Chhatrapati Shivaji Maharaj\nInternational Airport (Terminal 1)\n20:30 hrs, 13 Jan 2025\nKolkata\nCCU - Netaji Subhash Chandra Bose\nInternational Airport\n23:05 hrs, 13 Jan 2025\n*Date of Booking23 Nov 2024 13:04\n6E 

In [5]:
embeddings = OllamaEmbeddings(model="all-minilm")

C:\Users\aichr\AppData\Local\Temp\ipykernel_13192\2529941669.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="all-minilm")


In [6]:
db = Chroma.from_documents(chunk, embeddings, persist_directory="vectorFinal_db")
db.persist()

: 